In [1]:
#!tf annotation/banks

In [3]:
#!tf HuygensING/suriano --tool=ner

In [4]:
from tf.app import use
import pandas as pd
import re
from docx import Document
import os
import pandas as pd
import hashlib

In [5]:
# getting the data
A = use("HuygensING/suriano", hoist=globals())
print(A)

**Locating corpus resources ...**

<tf.apps.HuygensING/suriano.app.TfApp object at 0x000002710CFE4A90>


In [6]:
# nodes overview
unique_node_types = sorted(set(A.api.F.otype.data))
print(unique_node_types)
print(sorted(A.api.Fall()))

['author', 'bibl', 'biblScope', 'body', 'cell', 'chunk', 'collection', 'correspAction', 'correspDesc', 'date', 'div', 'editor', 'ent', 'entity', 'file', 'fileDesc', 'folder', 'head', 'hi', 'idno', 'institution', 'ln', 'metamark', 'msDesc', 'msIdentifier', 'name', 'note', 'num', 'p', 'page', 'pb', 'profileDesc', 'ptr', 'publicationStmt', 'quote', 'ref', 'resp', 'respStmt', 'row', 'settlement', 'sourceDesc', 'table', 'teiHeader', 'text', 'title', 'titleStmt']
['after', 'chunk', 'corresp', 'date', 'editorNotes', 'eid', 'empty', 'ename', 'extraspace', 'facs', 'file', 'folder', 'id', 'is_meta', 'is_note', 'kind', 'n', 'otype', 'recipient', 'recipientLoc', 'ref', 'rend', 'rend_decoded', 'rend_displaced', 'rend_footnote', 'rend_superscript', 'role', 'sender', 'senderLoc', 'shelfmark', 'source', 'str', 'summary', 'target', 'type', 'unit', 'when']


In [7]:
# number of total letters
total_letters = len(A.api.F.otype.s("text"))  
print(f"Total number of letters: {total_letters}")

Total number of letters: 725


In [8]:
# getting all of the text out of the nodes
all_letters = A.api.F.otype.s("text")  # Get all "text" nodes
letters_data = []

for letter in all_letters:
    text_content = " ".join(A.api.F.text.v(w) for w in A.api.L.d(letter, "word"))
    letters_data.append({"Letter ID": letter, "Content": text_content})

# creating data frame
df = pd.DataFrame(letters_data)
# showing first few rows
print(df.head())  
# saving to local machine
df.to_csv("full_letters_text.csv", index=False)
print("Saved full letters dataset as 'full_letters_text.csv'")



   Letter ID Content
0    2101982        
1    2101983        
2    2101984        
3    2101985        
4    2101986        
Saved full letters dataset as 'full_letters_text.csv'


In [9]:
# understanding the node types
for node_type in ["chunk", "div", "p", "text", "t", "quote", "body"]: 
    sample_nodes = A.api.F.otype.s(node_type)[:10]  
    for node in sample_nodes:
        text = A.api.F.str.v(node)
        if text:
            print(f"Node Type: {node_type}, Node ID: {node}, Text: {text[:100]}...")


Node Type: t, Node ID: 1, Text: Letter...
Node Type: t, Node ID: 2, Text: 02.001...
Node Type: t, Node ID: 3, Text: date...
Node Type: t, Node ID: 4, Text: 1616...
Node Type: t, Node ID: 5, Text: -...
Node Type: t, Node ID: 6, Text: 07...
Node Type: t, Node ID: 7, Text: -...
Node Type: t, Node ID: 8, Text: 09...
Node Type: t, Node ID: 9, Text: Christofforo...
Node Type: t, Node ID: 10, Text: Suriano...


In [11]:
#first_letter_content = []
#first_letter_id = A.api.F.otype.s("text")[0]  # This grabs the first letter

# Get all "t" nodes inside the first letter
#for t_node in A.api.L.d(first_letter_id, "t"):  
#    text = A.api.F.str.v(t_node)  # Extract text
#    if text:
#        first_letter_content.append(text)

# Print the full text
#print("\n".join(first_letter_content) if first_letter_content else "No content found.")


In [12]:
#for t_node in A.api.L.d(first_letter_id, "t"):  
#    text = A.api.F.str.v(t_node)  # Extract text
#    print(f"Node ID: {t_node}, Text: {text}")


In [13]:
# printing formated text 
first_letter_id = A.api.F.otype.s("text")[0] 
first_letter_words = []

# removing metadata from letters
unwanted_words = {
    "corrected", "above", "crossed", "out", "result", "correction", "followed", "by", "text", "letter", 
    "secretarial", "added", "line", "insertion", "mark", "first", "received", "written", "document",
    "is", "the", "of", "a", "an", "other", "h", "non", "stata", "ll", "ss","Main"
}
for t_node in A.api.L.d(first_letter_id, "t"):  
    text = A.api.F.str.v(t_node).strip()  
    if text.lower() in unwanted_words:  
        continue  
    if len(text) < 3 and not re.match(r"[.,!?]", text):  
        continue  

    first_letter_words.append(text)
# joining words into normal format
formatted_text = " ".join(first_letter_words)
# restoring punctiaction
formatted_text = re.sub(r"\s([,.\'\":;!?])", r"\1", formatted_text)
# ensuring capitalization
formatted_text = re.sub(r'([.!?])\s+([a-z])', lambda x: x.group(1) + " " + x.group(2).upper(), formatted_text)
formatted_text = formatted_text[0].upper() + formatted_text[1:] if formatted_text else formatted_text
# Print the cleaned and properly formatted letter
print(formatted_text if formatted_text else "No content found.")

Main Prima Serenissimo Principe nel venirmene questa città, ove arrivai lunedì, conve nuto metter qualche giorno più nel camino quello, che haverei desiderato per rispetto, che scrissi alla Serenità vostra nell ultime lettere col clarissimo Padavino Zurich. Gionto qui trovai clarissimo signor Luca Tron devotissimo figliolo quella serenissima Republica, quale con termine grande amore verso Patria, con affetto tutto intento servitio lei favorì assistermi tutte quelle cose, che pregai per publico servitio. Feci saper questo Principe mia venuta, commissione, che havevo trovarmi seco. Sua Altezza mandò immediate per doi gentilhuomini corte salutarmi, rallegrarsi del mio buon arrivo qui, dicendo, che mattina seguente haverebbe ascoltato. Mentre stavo atten dendo otto hore, che furono appuntate per confe rirmi palazzo signor Lamingher cameriero secreto, consigliero dell Altezza sua col mezo del medesimo clarissimo Tron fece sapere, che havendo serenissimo Leopoldo fatto intender qui, che sare

In [14]:
# extracting the italic text
def extract_italic_text(doc_path):
    doc = Document(doc_path)
    italic_texts = []
    for para in doc.paragraphs:
        for run in para.runs:
            if run.italic:
                italic_texts.append(run.text)
    return " ".join(italic_texts)
    
# directory
folder_path = "C:\\Users\\Kvrgic\\Desktop\\Soriano\\Soriano_texts"

def get_hash(text):
    normalized = text.lower().strip()
    normalized = " ".join(normalized.split()) 
    return hashlib.md5(normalized.encode("utf-8")).hexdigest()

# MAIN
data_italic = []
seen_hashes = set()

for filename in sorted(os.listdir(folder_path)):
    if filename.endswith(".docx"):
        file_path = os.path.join(folder_path, filename)
        italic_text = extract_italic_text(file_path)
        cleaned_text = italic_text.strip()
        if cleaned_text:
            text_hash = get_hash(cleaned_text)
            if text_hash not in seen_hashes:
                data_italic.append(cleaned_text)
                seen_hashes.add(text_hash)



df = pd.DataFrame(data_italic, columns=["text"])
df.to_csv("italic_texts_dataset.csv", index=False)


In [15]:
# printing italic data
print(data_italic)

['Traccia di sigillo Regesto antico Traccia di sigillo Regesto antico Traccia di sigillo Regesto antico Traccia di sigillo Regesto antico Traccia di sigillo Regesto antico Regesto antico Sigillo  di  maniera che l’opponersi tutti a così fatti  | pensieri sarà proprio della prudenza de’ principi | che ne sono interessati, quando fossero tutti | li prencipi dell’Unione con la serenissima Republica et li | stati di Olanda, et forse anco il re della | Gran Bertagna  tenuto da’ | Spagnuoli con una certa intelligenza di lega | o sia di buona corispon- denza; ma tutto | però con obligo della commune diffesa | ognuno per la parte, che potrà, o con danari | o con genti colla diversione, o in altra | maniera, sarebbe un metter la briglia | allo Spagnolo, non per una volta, ma per | sempre, che non saprebbe come reggersi | onde per consequenza si fermarebbe un giorno | dal voler abbracciar il mondo tutto   il mio Prencipe è inclinatissimo a questa unione | et  ognuno per la propria, et commune di

In [17]:
# getting all text nodes
all_letters = A.api.F.otype.s("text")
# making sure there are at least 34 letters
if len(all_letters) >= 34:
    letter_32_id = all_letters[4] 
    letter_words = []
    for t_node in A.api.L.d(letter_32_id, "t"):  
        text = A.api.F.str.v(t_node).strip()
        unwanted_words = {
            "corrected", "above", "crossed", "out", "result", "correction", "followed", "by", "text", "letter", 
            "secretarial", "added", "line", "insertion", "mark", "first", "received", "written", "document",
            "main","of"
        }
        if text.lower() in unwanted_words:
            continue
        if re.match(r"^[^a-zA-ZÀ-ÿ]+$", text):  
            continue
        letter_words.append(text)
    formatted_text = " ".join(letter_words)
    formatted_text = re.sub(r"\s([,.\'\":;!?])", r"\1", formatted_text)
    formatted_text = re.sub(r'([.!?])\s+([a-z])', lambda x: x.group(1) + " " + x.group(2).upper(), formatted_text)
    formatted_text = formatted_text[0].upper() + formatted_text[1:] if formatted_text else formatted_text
    print(formatted_text if formatted_text else "No content found.")
else:
    print("Letter not found")


Da  Serenissimo Principe il colonnello Sciombergh col quale ho riverentemente accennato nelle mie prime haver havuto ragionamento nell uscir dalla visita della serenissima Principessa mi venne ad incon trare et disse che doppo l desinare si saressimo trovati insieme perché voleva communicarmi qualche cosa stimata da lui di non poco rilevo Così fossimo insieme per buon spatio d hora nel qual tempo mi disse che haveva inteso dal signor Principe ciò che gl havevo con ferito et veduto anco quanto gl havevo fatto haver in scritto toccante lo stato del negotio la condotta sua et sopra che al presente pendi la controversia che tutto stava bene et quanto alla ragione qui ella era conosciuta per la serenissima Republica perché la dichia ratione sua fondava nell honesto né arciducali potevano dolersi ma aggiunse quanto a me dirò liberamente il termine ufficioso che usa la serenissima Republica con li principi dell Unione et la dimostratione di confidenza con loro nel farli communicar lo stato de